In [1]:
num = [
    "accommodates",
    "availability_30",
    "availability_365",
    "availability_60",
    "availability_90",
    "bathrooms",
    "bedrooms",
    "beds",
    "calculated_host_listings_count",
    "cleaning_fee",
    "extra_people",
    "guests_included",
    "host_has_profile_pic",
    "host_identity_verified",
    "host_is_superhost",
    "host_response_time",
    "latitude",
    "longitude",
    "maximum_nights",
    "minimum_nights",
    "number_of_reviews",
    "review_scores_accuracy",
    "review_scores_checkin",
    "review_scores_cleanliness",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_rating",
    "review_scores_value",
    "reviews_per_month",
    "security_deposit",
]

cat = [
    "bed_type",
    "cancellation_policy",
    "host_verifications_email",
    "host_verifications_facebook",
    "host_verifications_google",
    "host_verifications_government_id",
    "host_verifications_identity_manual",
    "host_verifications_jumio",
    "host_verifications_kba",
    "host_verifications_manual_offline",
    "host_verifications_manual_online",
    "host_verifications_offline_government_id",
    "host_verifications_phone",
    "host_verifications_reviews",
    "host_verifications_selfie",
    "host_verifications_sent_id",
    "host_verifications_sesame",
    "host_verifications_sesame_offline",
    "host_verifications_weibo",
    "host_verifications_work_email",
    "host_verifications_zhima_selfie",
    "instant_bookable",
    "is_location_exact",
    "license",
    "require_guest_phone_verification",
    "require_guest_profile_picture",
    "room_type",
]

string = [
    "access",
    "amenities",
    "calendar_updated",
    "city",
    "description",
    "first_review",
    "host_about",
    "host_location",
    "host_neighborhood",
    "host_response_rate",
    "host_since",
    "host_verifications",
    "house_rules",
    "interaction",
    "last_review",
    "name",
    "neighborhood",
    "neighborhood_overview",
    "property_type",
    "smart_location",
    "space",
    "state",
    "street",
    "suburb",
    "summary",
    "transit",
    "zipcode",
]
len(num + cat)

57

In [2]:
from datasets import Dataset, DatasetDict, load_dataset
from auto_mm_bench.datasets import dataset_registry
from sklearn.preprocessing import OrdinalEncoder
from src.dataset_info import get_dataset_info

/home/james/CodingProjects/TextNTabularExplanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from datasets import load_from_disk, load_dataset
import wandb
import os
import yaml
import argparse
from transformers.trainer_callback import EarlyStoppingCallback
import evaluate
import numpy as np
from lion_pytorch import Lion
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim import AdamW
from src.utils import (
    prepare_text,
    row_to_string,
    multiple_row_to_string,
)
from src.dataset_info import get_dataset_info


In [3]:
dataset_name = "melbourne_airbnb"

train_dataset = dataset_registry.create(dataset_name, "train")
test_dataset = dataset_registry.create(dataset_name, "test")
cols = train_dataset.feature_columns + train_dataset.label_columns

train_txt = train_dataset.data[cols]
test_txt = test_dataset.data[cols]

# load dataset from dataframe
train_ds = Dataset.from_pandas(train_txt)
train_ds = train_ds.class_encode_column(train_dataset.label_columns[0])
test_ds = Dataset.from_pandas(test_txt)
test_ds = test_ds.class_encode_column(train_dataset.label_columns[0])


In [10]:
model_base = "microsoft/deberta-v3-base"
dataset_name = "jigsaw_unintended_bias100K"


di = get_dataset_info(dataset_name)
train_dataset = dataset_registry.create(dataset_name, "train")

test_dataset = dataset_registry.create(dataset_name, "test")
cols = train_dataset.feature_columns + train_dataset.label_columns

train_txt = train_dataset.data[cols]
test_txt = test_dataset.data[cols]

# load dataset from dataframe
train_ds = Dataset.from_pandas(train_txt)
train_ds = train_ds.class_encode_column(train_dataset.label_columns[0])
test_ds = Dataset.from_pandas(test_txt)
test_ds = test_ds.class_encode_column(train_dataset.label_columns[0])

train_ds = train_ds.train_test_split(
    test_size=0.15, seed=42, stratify_by_column=train_dataset.label_columns[0]
)

ds = DatasetDict(
    {"train": train_ds["train"], "validation": train_ds["test"], "test": test_ds}
)

# Now we have made the split but still need to deal with missing values, and that depends on the column type

# All as text
train_all_text = ds["train"].to_pandas()
val_all_text = ds["validation"].to_pandas()
test_all_text = ds["test"].to_pandas()

train_all_text[train_dataset.feature_columns] = train_all_text[
    train_dataset.feature_columns
].astype("str")
val_all_text[train_dataset.feature_columns] = val_all_text[
    train_dataset.feature_columns
].astype("str")
test_all_text[train_dataset.feature_columns] = test_all_text[
    train_dataset.feature_columns
].astype("str")

ds_all_text = DatasetDict(
    {
        "train": Dataset.from_pandas(train_all_text),
        "validation": Dataset.from_pandas(val_all_text),
        "test": Dataset.from_pandas(test_all_text),
    }
)

# Dataset
di = get_dataset_info(dataset_name, model_type="all_as_text")
# dataset = load_dataset(di.ds_name) # , download_mode="reuse_dataset_if_exists")
dataset = ds_all_text
dataset = prepare_text(dataset, "all_as_text", dataset_name)
if di.prob_type == "regression":
    mean_price = np.mean(dataset["train"]["label"])
    std_price = np.std(dataset["train"]["label"])

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    model_base, num_labels=di.num_labels, problem_type=di.prob_type
)
tokenizer = AutoTokenizer.from_pretrained(model_base)

# Tokenize the dataset


def encode(examples):
    return {
        "label": np.array([examples[di.label_col]]),
        **tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            # max_length=args["max_length"],
        ),
    }


dataset = dataset.map(encode, load_from_cache_file=False)

# # Fast dev run if want to run quickly and not save to wandb
# if args["fast_dev_run"]:
#     args["num_epochs"] = 1
#     args["tags"].append("fast-dev-run")
#     dataset["train"] = dataset["train"].select(range(500))
#     dataset["test"] = dataset["test"].select(range(10))
#     output_dir = os.path.join(args["output_root"], "testing")
#     print(
#         "\n######################    Running in fast dev mode    #######################\n"
#     )

No model type specified for jigsaw_unintended_bias100K. (This is fine during dataset creation)
No model type specified for jigsaw. (This is fine during dataset creation)                


Using dataset jigsaw_unintended_bias100K, all as text version


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [11]:
lens = [len(i) for i in dataset["train"]["input_ids"]]


In [15]:
len([l for l in lens if l > 512]) / len(lens)


5.882352941176471e-05

In [4]:
model_base = "distilroberta-base"
dataset_name = "airbnb"


# di = get_dataset_info(dataset_name)
# train_dataset = dataset_registry.create(dataset_name, "train")

# test_dataset = dataset_registry.create(dataset_name, "test")
# cols = train_dataset.feature_columns + train_dataset.label_columns

# train_txt = train_dataset.data[cols]
# test_txt = test_dataset.data[cols]

# # load dataset from dataframe
# train_ds = Dataset.from_pandas(train_txt)
# train_ds = train_ds.class_encode_column(train_dataset.label_columns[0])
# test_ds = Dataset.from_pandas(test_txt)
# test_ds = test_ds.class_encode_column(train_dataset.label_columns[0])

# train_ds = train_ds.train_test_split(
#     test_size=0.15, seed=42, stratify_by_column=train_dataset.label_columns[0]
# )

# ds = DatasetDict(
#     {"train": train_ds["train"], "validation": train_ds["test"], "test": test_ds}
# )

# # Now we have made the split but still need to deal with missing values, and that depends on the column type

# # All as text
# train_all_text = ds["train"].to_pandas()
# val_all_text = ds["validation"].to_pandas()
# test_all_text = ds["test"].to_pandas()

# train_all_text[train_dataset.feature_columns] = train_all_text[
#     train_dataset.feature_columns
# ].astype("str")
# val_all_text[train_dataset.feature_columns] = val_all_text[
#     train_dataset.feature_columns
# ].astype("str")
# test_all_text[train_dataset.feature_columns] = test_all_text[
#     train_dataset.feature_columns
# ].astype("str")

# ds_all_text = DatasetDict(
#     {
#         "train": Dataset.from_pandas(train_all_text),
#         "validation": Dataset.from_pandas(val_all_text),
#         "test": Dataset.from_pandas(test_all_text),
#     }
# )

# Dataset
di = get_dataset_info(dataset_name, model_type="all_as_text")
# , download_mode="reuse_dataset_if_exists")
dataset = load_dataset(di.ds_name)
# dataset = ds_all_text
dataset = prepare_text(dataset, "all_as_text", dataset_name)
if di.prob_type == "regression":
    mean_price = np.mean(dataset["train"]["label"])
    std_price = np.std(dataset["train"]["label"])

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    model_base, num_labels=di.num_labels, problem_type=di.prob_type
)
tokenizer = AutoTokenizer.from_pretrained(model_base)

# Tokenize the dataset


def encode(examples):
    return {
        "label": np.array([examples[di.label_col]]),
        **tokenizer(
            examples["text"],
            # truncation=True,
            # padding="max_length",
            # max_length=args["max_length"],
        ),
    }


dataset = dataset.map(encode)  # , load_from_cache_file=False)

# # Fast dev run if want to run quickly and not save to wandb
# if args["fast_dev_run"]:
#     args["num_epochs"] = 1
#     args["tags"].append("fast-dev-run")
#     dataset["train"] = dataset["train"].select(range(500))
#     dataset["test"] = dataset["test"].select(range(10))
#     output_dir = os.path.join(args["output_root"], "testing")
#     print(
#         "\n######################    Running in fast dev mode    #######################\n"
#     )


Using dataset airbnb, all as text version


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--melbourne_airbnb_all_text-ca650dbf1ce581bf/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 372.46it/s]
No model type specified for airbnb. (This is fine during dataset creation)
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassifica

In [5]:
lens = [len(i) for i in dataset["train"]["input_ids"]]


In [6]:
len([l for l in lens if l > 512]) / len(lens)


0.40364850976361766

In [7]:
lens

[455,
 479,
 539,
 429,
 492,
 430,
 488,
 532,
 456,
 501,
 474,
 586,
 469,
 443,
 541,
 469,
 634,
 493,
 377,
 498,
 598,
 581,
 409,
 507,
 399,
 537,
 650,
 489,
 454,
 455,
 548,
 546,
 448,
 579,
 483,
 377,
 437,
 366,
 455,
 703,
 522,
 404,
 681,
 509,
 400,
 521,
 513,
 807,
 493,
 530,
 550,
 500,
 449,
 556,
 1341,
 607,
 394,
 477,
 474,
 538,
 518,
 400,
 631,
 557,
 484,
 524,
 476,
 524,
 476,
 593,
 469,
 453,
 477,
 471,
 480,
 442,
 592,
 480,
 474,
 483,
 536,
 703,
 575,
 643,
 521,
 458,
 485,
 553,
 479,
 440,
 555,
 636,
 541,
 916,
 500,
 469,
 466,
 546,
 601,
 440,
 544,
 533,
 406,
 432,
 548,
 476,
 351,
 492,
 430,
 506,
 428,
 693,
 529,
 567,
 458,
 547,
 593,
 392,
 549,
 546,
 380,
 426,
 527,
 552,
 438,
 472,
 407,
 368,
 447,
 464,
 432,
 452,
 445,
 457,
 423,
 443,
 489,
 570,
 349,
 344,
 487,
 424,
 446,
 414,
 445,
 633,
 495,
 447,
 442,
 390,
 347,
 370,
 491,
 449,
 419,
 593,
 502,
 516,
 461,
 332,
 534,
 563,
 611,
 576,
 581,
 660,
 59